In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument(r"--no-sandbox")
options.add_argument(r"--disable-dev-shm-usage")
options.add_argument(r"--disable-gpu")

# point directly to the cloned profile
options.add_argument(r"--user-data-dir=C:\SeleniumProfiles")
options.add_argument(r"--profile-directory=Profile17")

path = r"C:\Users\rando\chromedriver-win64\chromedriver.exe"

driver = webdriver.Chrome(service=Service(path), options=options)
driver.get("https://www.cogencis.com/")

print(driver.title)

driver.quit()

Fund Management | Wealth Management | NSE Cogencis WorkStation


In [47]:
%load_ext autoreload
%autoreload 2
import time, base64, os, re,json5, pprint,random , math, hashlib, inspect
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
import pandas as pd
import numpy as np
from io import StringIO

from app.operation_executor import OperationExecutor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
{
    "primary":["sha1","sha256"], # packet will now have original, sha1, sha256
    "secondary":{
        "sha1":"pandas_df", #packet will be prev + sha1_pandas_df, sha1_something
        "sha1":"something", 
    },
    "tertiary":{
        "pandas_df":"sha512" #packet will be 
    }
}

In [ ]:
html_path = """<table><tbody><tr><td rowspan="3">Period / Tenor of Deposit </td><td colspan="2">Less than Rs.3 crore</td></tr><tr><td>Existing</td><td>Revised w.e.f 01.08.2025</td></tr><tr><td>Rate(% per annum )</td><td>Rate(% per annum )</td></tr><tr><td>7 days to 14 days</td><td>2.80</td><td>2.80</td></tr><tr><td>15 days to 29 days</td><td>2.80</td><td>2.80</td></tr><tr><td>30 days to 45 days</td><td>3.00</td><td>3.00</td></tr><tr><td>46 days to 90 days</td><td>3.25</td><td>3.25</td></tr><tr><td>91 days to 120 days</td><td>3.50</td><td>3.50</td></tr><tr><td>121 days to 180 days</td><td>3.85</td><td>3.85</td></tr><tr><td>181 days to less than 9 months</td><td>4.50</td><td>4.50</td></tr><tr><td>9 months to less than 1 year</td><td>4.75</td><td>4.75</td></tr><tr><td>1 year</td><td>6.10</td><td>6.10</td></tr><tr><td>Above 1 year to less than 2 years( Except 444 &amp; 555 Days)</td><td>6.60</td><td>6.50</td></tr><tr><td>444 days(Ind Secure Product)</td><td>6.90</td><td>6.70</td></tr><tr><td>555 days( Ind Green Product)</td><td>6.80</td><td>6.60</td></tr><tr><td>2 years to less than 3 years</td><td>6.40</td><td>6.40</td></tr><tr><td>3 years to less than 5 years</td><td>6.25</td><td>6.25</td></tr><tr><td>5 year</td><td>6.00</td><td>6.00</td></tr><tr><td>Above 5 years</td><td>6.00</td><td>6.00</td></tr></tbody></table>"""
            

def _generalize_table_df(html_string, MAX_COLUMN=15):
    cols = [f"column_{i}" for i in range(1, MAX_COLUMN + 1)]
    dfs = pd.read_html(StringIO(html_string), flavor='html5lib')

    all_dfs = []
    for df in dfs:
        if df.empty:
            continue
        
        df = df.iloc[:, :MAX_COLUMN].copy() #truncate if MAX_LENGTH
        df.columns = cols[:df.shape[1]]
        for i in range(df.shape[1], MAX_COLUMN):df[f"column_{i+1}"] = ""
        
        df = df.reindex(columns=cols)
        all_dfs.append(df)
    #     return pd.concat(all_dfs, ignore_index=True)
    # return pd.DataFrame(columns=cols)
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        return final_df.to_csv(index=False, sep=',', lineterminator='\n')
    else:
        return pd.DataFrame(columns=cols).to_csv(index=False, sep=',', lineterminator='\n')

def _generate_hash_sha1(text:str)->str:
    if not isinstance(text,str):
        return f"{inspect.currentframe().f_code.co_name}: input non str"
    return hashlib.sha1(text.encode()).hexdigest()

        
dfs = _generalize_table_df(html_path)

hash = _generate_hash_sha1(dfs)

In [57]:
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\cache_21082025_170104.json5"

data = Helper.load_json(path, typ="json5")

ops = OperationExecutor()
function_to_execute= {
    # "sha256":"_generate_hash_sha256",
    # "sha1":"_generate_hash_sha1",
    "normal":"_generalize_table_df"
}

processed_data = ops.runner(data,function_to_execute)

Helper.save_json(processed_data,"process.json")


Processing for Bank :Bank of Baroda


Processing for Bank :Bank of India


Processing for Bank :Bank of Maharashtra


Processing for Bank :Canara Bank


Processing for Bank :Central Bank of India


Processing for Bank :Indian Bank


Processing for Bank :Indian Overseas Bank


Processing for Bank :Punjab and Sind Bank


Processing for Bank :Punjab National Bank


Processing for Bank :State Bank of India


Processing for Bank :Uco Bank


Processing for Bank :Union Bank of India



In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import random, time, math, ssl

ssl._create_default_https_context = ssl._create_stdlib_context

def human_scroll(driver, total_scroll=2000, step=50):
    for y in range(0, total_scroll, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(random.uniform(0.05, 0.3))


def human_move_mouse(driver, start=(0, 0), end=(300, 300), steps=30):
    actions = ActionChains(driver)
    x1, y1 = start
    x2, y2 = end
    for t in range(steps):
        progress = t / steps
        x = x1 + (x2 - x1) * progress + random.uniform(-2, 2)
        y = y1 + (y2 - y1) * progress + math.sin(progress * math.pi) * 20
        actions.move_by_offset(int(x), int(y))
    actions.perform()

def human_click(elem):
    time.sleep(random.uniform(0.2, 1.5))
    elem.click()

profile_path = Helper.load_json("paths.json").get("profile_path")
profile_dir = Helper.load_json("paths.json").get("profile_name")

options = Options()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

human_scroll(driver, total_scroll=1500, step=70)
human_move_mouse(driver, start=(0, 0), end=(200, 200), steps=40)

try:
    table = driver.find_element(By.TAG_NAME, "table")
    human_click(table) # Just demo click
    print("Table found:", table.get_attribute("outerHTML")[:200])
except Exception as e:
    print("No table found:", e)

time.sleep(5)
driver.quit()

# import undetected_chromedriver as uc 

# profile_path = r"C:\Users\kaustubh.keny\AppData\Local\Google\Chrome\User Data"
# profile_dir = "Default" # Change to match your actual one

# options = uc.ChromeOptions()
# options.add_argument(f"--user-data-dir={profile_path}")
# options.add_argument(f"--profile-directory={profile_dir}")
# # options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--start-maximized")

# driver = uc.Chrome(driver_executable_path=r"C:\Users\kaustubh.keny\chromedriver-win64\chromedriver.exe",options=options)
# driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

# import scrapy

# class UnionBankSpider(scrapy.Spider):
#     name = "unionbank_tables"
#     allowed_domains = ["unionbankofindia.co.in"]
#     start_urls = ["https://www.unionbankofindia.co.in/en/details/rate-of-interest"]

#     def parse(self, response):
#         tables = response.css("table").getall()
#         self.logger.info(f"Found {len(tables)} tables")
#         for idx, table in enumerate(tables):
#             yield {
#                 "table_index": idx,
#                 "html": table
#             }



: 